In [88]:
import pandas as pd
import numpy as np

In [108]:
data = pd.read_csv('weather_data_days.csv', sep=',')
data = data.drop(['day_end'], axis='columns')
data = data.drop(data[data['day_start'] < 1].index)

In [110]:
X= data.drop('day_start', axis=1)
y = data['day_start'].astype('int')

In [111]:
X.shape, y.shape

((3055, 61), (3055,))

In [112]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=17)

In [113]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=17)

In [114]:
X_train.shape, X_valid.shape

((2138, 61), (917, 61))

In [115]:
from sklearn.model_selection import  cross_val_score
np.mean(cross_val_score(dtc, X_train, y_train, cv=5))

c:\users\user\projects\pogoda_news_parser\venv\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


0.8316323403882772

In [96]:
%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(
    n_estimators=47, random_state=17,
    n_jobs=-1, max_features=2,
    )
rf.fit(X_train, y_train)

Wall time: 0 ns


RandomForestClassifier(max_features=2, n_estimators=47, n_jobs=-1,
                       random_state=17)

In [97]:
%time
rf_valid_pred = rf.predict(X_valid)

Wall time: 0 ns


In [98]:
from sklearn.metrics import accuracy_score
accuracy_score(y_valid, rf_valid_pred)

0.8704177323103154

In [99]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score

In [100]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [101]:
rfc = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)

In [102]:
results = cross_val_score(rfc, X, y, cv=skf)

In [103]:
print("CV accuracy score: {:.2f}%".format(results.mean()*100))

CV accuracy score: 87.07%


In [27]:
# Инициализируем валидацию
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [105]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
trees_grid = range(20,65) 

In [106]:
# Обучаем на тренировочном датасете
for ntrees in trees_grid:
    rfc = RandomForestClassifier(n_estimators=ntrees, random_state=42, n_jobs=-1, oob_score=True)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

c:\users\user\projects\pogoda_news_parser\venv\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\user\projects\pogoda_news_parser\venv\lib\site-packages\sklearn\ensemble\_forest.py:544: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /
c:\users\user\projects\pogoda_news_parser\venv\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\user\projects\pogoda_news_parser\venv\lib\site-packages\sklearn\ensemble\_forest.py:544: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /
c:\users\user\projects\pogoda_news_parser\venv\lib\site-packages\sklearn\ens

In [107]:
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} trees".format(max(test_acc.mean(axis=1))*100, 
                                                        trees_grid[np.argmax(test_acc.mean(axis=1))]))

Best accuracy on CV is 87.11% with 53 trees


In [33]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
max_depth_grid = range(80,120)

In [34]:
# Обучаем на тренировочном датасете
for max_depth in max_depth_grid:
    rfc = RandomForestClassifier(n_estimators=91, random_state=42, n_jobs=-1, oob_score=True, max_depth=max_depth)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(start_x, start_y):
        X_train, X_test = start_x.iloc[train_index], start_x.iloc[test_index]
        y_train = start_y.iloc[train_index]
        y_test = start_y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} max_depth".format(max(test_acc.mean(axis=1))*100, 
                                                        max_depth_grid[np.argmax(test_acc.mean(axis=1))]))

Best accuracy on CV is 88.39% with 94 max_depth


In [35]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
min_samples_leaf_grid = [1, 3, 5, 7, 9, 11, 13, 15, 17, 20, 22, 24]

In [36]:
# Обучаем на тренировочном датасете
for min_samples_leaf in min_samples_leaf_grid:
    rfc = RandomForestClassifier(n_estimators=91, random_state=42, n_jobs=-1, 
                                 oob_score=True, min_samples_leaf=min_samples_leaf, max_depth=94)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(start_x, start_y):
        X_train, X_test = start_x.iloc[train_index], start_x.iloc[test_index]
        y_train = start_y.iloc[train_index]
        y_test = start_y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} min_samples_leaf".format(max(test_acc.mean(axis=1))*100, 
                                                                min_samples_leaf_grid[np.argmax(test_acc.mean(axis=1))]))

Best accuracy on CV is 88.39% with 1 min_samples_leaf


In [37]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
max_features_grid = [2, 4, 6, 8, 10, 12, 14, 16]

# Обучаем на тренировочном датасете
for max_features in max_features_grid:
    rfc = RandomForestClassifier(n_estimators=91, random_state=42, n_jobs=-1, 
                                 oob_score=True, max_features=max_features)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(start_x, start_y):
        X_train, X_test = start_x.iloc[train_index], start_x.iloc[test_index]
        y_train = start_y.iloc[train_index]
        y_test = start_y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} max_features".format(max(test_acc.mean(axis=1))*100, 
                                                        max_features_grid[np.argmax(test_acc.mean(axis=1))]))

Best accuracy on CV is 88.31% with 10 max_features
